In [1]:
from dask import delayed

In [2]:
%load_ext cython

In [3]:
# Create a toy Cython function and put it into a module named remoteCython

In [4]:
%%cython -n remoteCython

def cython_sum(int a, int b):
    return a+b

In [5]:
# Set up a distributed cluster (minimal, just for illustration)
# I use SLURM.
from dask_jobqueue import SLURMCluster
from distributed import Client

In [7]:
cluster = SLURMCluster('ssrde.ucsd.edu:6818',
                        memory="1GB",
                       processes=1,
                       cores=1,
)


/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [6]:
from distributed import Client, LocalCluster

In [7]:
cluster = LocalCluster()

In [8]:
#cluster.start_workers(1)   # Start as many workers as needed.

In [13]:
import IPython
ipython_cache_dir = IPython.paths.get_ipython_cache_dir()
ipython_cache_dir

'/local/home/tsainbur/.cache/ipython'

In [10]:
client = Client(cluster)

In [11]:
def init_pyx(dask_worker):
    import pyximport
    pyximport.install()

    import sys
    sys.path.insert(0,'{}/cython/'.format(IPython))   # <<< replace <IPYTHONCACHEDIR> as appropriate

    import remoteCython


In [12]:
client.register_worker_callbacks(init_pyx)  # This runs init_pyx() on any Worker at init

ModuleNotFoundError: No module named 'remoteCython'

In [23]:
import remoteCython

In [24]:
# ASIDE: you can find where the full path of Cython-generated library by
# looking at remoteCython.__file__

# The following creates a task and submits to the scheduler.
# The task computes the sum of 123 and 321 via the Cython function defined above
future = client.compute(delayed(remoteCython.cython_sum)(123,321)) 

In [25]:
# The task is executed on the remote worker
# We fetch the result from the remote worker
print(future.result())   # This prints 444

KeyboardInterrupt: 

In [8]:
# We're done. Let's release the SLURM jobs.
cluster.close()

NameError: name 'cluster' is not defined